# Exemples et tests d'utilisation du systeme de Primitive de PyPot
* [Documentation sur les primitives](http://nbviewer.ipython.org/github/poppy-project/pypot/blob/master/samples/notebooks/Controlling%20a%20Poppy%20humanoid%20in%20V-REP%20using%20pypot.ipynb#Using-primitives)
* [Exemples d'utilisation des primitives (notebook)](http://nbviewer.ipython.org/github/poppy-project/pypot/blob/master/samples/notebooks/Controlling%20a%20Poppy%20humanoid%20in%20V-REP%20using%20pypot.ipynb#Using-primitives)
* [Primitive de bases inclues dans pypot (github)](https://github.com/poppy-project/pypot/tree/master/pypot/primitive)
* [Primitives inclues dans Poppy_humanoïde](https://github.com/poppy-project/poppy-humanoid/tree/master/software/poppy_humanoid/primitives)

In [ ]:
from pprint import pprint


In [ ]:
#imports and initilaize virutal poppy using vrep
from pypot.vrep import from_vrep
from poppy.creatures import PoppyHumanoid
robot = PoppyHumanoid(simulator='vrep')

In [ ]:
#import and initialize physical poppy
from poppy.creatures import PoppyHumanoid
robot = PoppyHumanoid()

In [ ]:
#Rend le robot mou et deconnecte les moteurs
for m in robot.motors:
    m.compliant = True
robot.close()

## Primitives de sécurité
Activation des deux primitives de sécurités fournies avec la poppy_humanoïde, elle permettent d'eviter les dmages sur les moteurs, notement dus a leur surchauffe.
* **LimitTorque** : limite le couple appliqué sur le les moteurs, evite leur surcauffe mais aussi de casser des pieces.
* **TemperatureMonitor** : Surveille la temperature des moteur, joue un son et affiche une alerte lorsque la temperature de certains moteurs atteind la limite fixée.

Ces deux primitives devraient être activées au debut de nimporte quelle utilisation de poppy.

In [ ]:
from poppy_humanoid.primitives.safe import LimitTorque
plimit_torque = LimitTorque(robot)
plimit_torque.start()

In [ ]:
from poppy_humanoid.primitives.safe import TemperatureMonitor
ptemp_monitor = TemperatureMonitor(robot,temp_limit=50, sound="/opt/sounds/ouch.wav")
ptemp_monitor.start()

### Moniteur de temperature custom
Le moniteur de temperature fournis est sympa mais si pour une raison 
ou une autre l'opérateur de Poppy de vois/entend pas l'alerte cela risque de causer des
domages irreversible sur les moteurs. Cette version propose de rendre rendre "mou" puis totalement inactif
les moteurs qui seraient chaud depuis trop longtemps.

⇒ La classe est définie dans [poppy_rate/primitives/safety.py](/edit/poppy_rate/primitives/safety.py) 

Remarque : limiter le couple des moteur ne semble pas reelement utile, simplement rendre les moteurs compliant si ce n'est pas deja le cas devrait suffire.

In [ ]:
from poppy_rate.primitives.safety import CustomTemperatureMonitor
robot.attach_primitive(CustomTemperatureMonitor(robot,temp_limit=50, sound="/opt/sounds/ouch.wav"),"temp_monitor")
robot.temp_monitor.start()

### Loggeur de temperatures min/max
Primitive permettant d'afficher un graph de l'évolution des temperature min/max des moteurs de poppy

⇒ La classe est définie dans [poppy_rate/primitives/safety.py](/edit/poppy_rate/primitives/safety.py)

In [ ]:
from poppy_rate.primitives.safety import TemperatureLogger

In [ ]:
temp_logger = TemperatureLogger(robot)
temp_logger.start()

In [ ]:
%pylab inline

rcParams['figure.figsize'] = 15, 4

t = linspace(0, 20, len(temp_logger.temp_max))
plot(t, temp_logger.temp_min,'g:')
plot(t, temp_logger.temp_max,'r-')

legend(('min', 'max'))

In [ ]:
import operator
# liste triée par t° de tuples (nom_moteur,t°)
temperatures =sorted ([ (m.name,m.present_temperature) for m in robot.motors ] , key=operator.itemgetter(1),reverse=True)
#affiche le tout
for m,t in temperatures:
    print (u"%20s' :   %5s°C") % (m,t)

## Positions prédéfinies
### Assis

In [ ]:
from poppy_humanoid.primitives.posture import SitPosition
robot.attach_primitive(SitPosition(robot),"sit")

In [ ]:
robot.sit.start()

In [ ]:
robot.sit.stop()

### Debout

In [ ]:
from poppy_humanoid.primitives.posture import StandPosition
robot.attach_primitive(StandPosition(robot),"stand")

In [ ]:
robot.stand.start()

##Comportement Custom
### Coucou

In [ ]:
for p in ['r','l']:
    for m in ['%s_elbow_y'%p, '%s_shoulder_y'%p,'%s_arm_z'%p]:
        robot.__dict__.get(m).compliant = False

    robot.goto_position({'%s_elbow_y'%p:-90, '%s_shoulder_y'%p:-75,'%s_arm_z'%p:0}, 1, wait=True)
    for k in range(3):
        robot.goto_position({'%s_arm_z'%p:45}, 0.7, wait=True)
        robot.goto_position({'%s_arm_z'%p:-45}, 0.7, wait=True)
        

    robot.goto_position({'%s_elbow_y'%p:0, '%s_shoulder_y'%p:0,'%s_arm_z'%p:0}, 3, wait=True)

    for m in robot.motors:
        m.compliant = True

Fait coucou à la mode Elisabeth II, bras droit puis bras gauche :
(repris dans [poppy_rate/primitives/behaviour.py](/edit/poppy_rate/primitives/behaviour.py) ) :

*N'a pas l'air de fonctionner dans poppy_vrep mais pourtant fonctionnait bien quand testé sur poppy physique*

In [ ]:
arm = {'r':90,'l':-90}
shoulder = {'r':-40,'l':40}
shoulder2 = {'r':-15,'l':15}
for p in ['r','l']:
    for m in ['%s_elbow_y'%p, '%s_shoulder_y'%p,'%s_shoulder_x'%p,'%s_arm_z'%p]:
        robot.__dict__.get(m).compliant = False

    robot.goto_position({'%s_elbow_y'%p:-50, '%s_shoulder_y'%p:-165,'%s_arm_z'%p:arm[p]}, 1.5, wait=True)

    for k in range(3):
        robot.goto_position({'%s_elbow_y'%p:-30,'%s_shoulder_x'%p:shoulder[p]}, 2, wait=True)
        robot.goto_position({'%s_elbow_y'%p:-65,'%s_shoulder_x'%p:0}, 2, wait=True)
    
    robot.goto_position({'%s_elbow_y'%p:0, '%s_shoulder_y'%p:0,'%s_arm_z'%p:0,'%s_shoulder_x'%p:shoulder2[p]}, 2, wait=True)
    for m in robot.motors:
        m.compliant = True